# 🦌 Logic Gate Symbol Library 🔣

`IPyElk` can serve as a jumpping off point for building different types of diagrams
based on defined shapes. This notebook demonstrates importing the logic gate symbol
library.

In [ ]:
import importnb
import networkx as nx

import ipyelk
import ipyelk.nx
import ipyelk.tools.tools
from ipyelk.contrib.library import logic_gates as logic
from ipyelk.diagram import layout_options as opt

# Logic Gates

https://upload.wikimedia.org/wikipedia/commons/c/cb/Circuit_elements.svg

In [ ]:
gate_library = logic.Gate.make_defs()

# Simple Graph to draw the Logic Gate Symbol Library

In [ ]:
def logic_library():
    # build graph
    symbols = [symbol() for symbol in logic.Gate.__subclasses__()]
    g = nx.Graph()
    tree = nx.DiGraph()

    for s in symbols:
        name = s.__class__.__name__
        g.add_node(name, **s.node(id=name).dict(exclude_none=True))

    # configure app
    app = ipyelk.Elk(
        transformer=ipyelk.nx.XELK(layouts={}, source=(g, None)),
        layout={"height": "100%"},
    )
    app.diagram.symbols = gate_library
    return app


if __name__ == "__main__":
    app = logic_library()
    display(app)

# 1-bit memory circuit

In [ ]:
sizing_opts = opt.OptionsWidget(
    options=[
        opt.NodeSizeConstraints(),
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
    ]
).value

HIGH = "on"
LOW = "off"


states = {
    ("nor 1", "a"): {
        "value": 0,
    },
    ("nor 1", "b"): {
        "value": 0,
    },
    ("nor 1", "out"): {
        "targets": [("nor 2", "b")],
        "value": 0,
    },
    ("nor 2", "a"): {
        "value": 0,
    },
    ("nor 2", "b"): {
        "value": 0,
    },
    ("nor 2", "out"): {
        "targets": [("one_b", "out"), ("nor 1", "a")],
        "value": 0,
    },
    "set": {
        "targets": [("one_b", "set")],
        "value": 0,
    },
    "reset": {
        "targets": [("one_b", "reset")],
        "value": 0,
    },
    ("one_b", "set"): {
        "targets": [("nor 1", "b")],
        "value": 0,
    },
    ("one_b", "reset"): {
        "targets": [("nor 2", "a")],
        "value": 0,
    },
    ("one_b", "out"): {
        "targets": [("output", None)],
        "value": 0,
    },
    "output": {
        "value": 0,
    },
}


def xor(a, b):
    return (a and not b) or (b and not a)


def step_state(states, set_, reset):
    nor1_a = states[("nor 1", "a")]["value"]
    nor2_a = states[("nor 2", "a")]["value"]

    nor1_b = set_
    nor2_b = reset
    nor2 = not (nor2_a or nor2_b)
    nor1 = not (nor1_a or nor1_b)

    output = nor2
    nor1_a = nor2
    nor2_a = nor1

    states[("nor 1", "a")]["value"] = nor1_a
    states[("nor 1", "b")]["value"] = nor1_b
    states[("nor 1", "out")]["value"] = nor1
    states[("nor 2", "a")]["value"] = nor2_a
    states[("nor 2", "b")]["value"] = nor2_b
    states[("nor 2", "out")]["value"] = nor2
    states["set"]["value"] = set_
    states[("one_b", "set")]["value"] = set_
    states[("one_b", "reset")]["value"] = reset
    states[("one_b", "out")]["value"] = output
    states["reset"]["value"] = reset
    states["output"]["value"] = output


def update_graph(states, g, init=False):
    for state, data in states.items():
        if isinstance(state, tuple):
            u, source_port = state
        else:
            u, source_port = state, None

        for i, target in enumerate(data.get("targets", [])):

            v, target_port = target

            if init:
                g.add_edge(
                    u,
                    v,
                    sourcePort=source_port,
                    targetPort=target_port,
                    properties={"cssClasses": HIGH if data.get("value") else LOW},
                )
            else:
                g.edges[u, v, 0]["properties"]["cssClasses"] = (
                    HIGH if data.get("value") else LOW
                )

        if not isinstance(state, tuple):
            g.nodes[state]["properties"]["cssClasses"] = (
                HIGH if data.get("value") else LOW
            )
        else:
            n, p = state
            ports = g.nodes[n].get("ports", [])
            for port in ports:
                if port.get("id", None) == f"{n}.{p}":
                    port["properties"] = {
                        "cssClasses": HIGH if data.get("value") else LOW
                    }


def onebit_memory_example():
    # build graph
    sizing_opts = opt.OptionsWidget(
        options=[
            opt.NodeSizeConstraints(),
            opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        ]
    ).value

    gates = [
        ("nor 1", logic.Nor_Gate()),
        ("nor 2", logic.Nor_Gate()),
    ]

    g = nx.MultiDiGraph()
    tree = nx.DiGraph()
    g.add_node(
        "one_b",
        labels=[{"id": "one_b_label_0", "text": "One Bit Memory"}],
        ports=[
            {
                "id": "one_b.set",
                "labels": [],
                "layoutOptions": {
                    "org.eclipse.elk.port.side": "WEST",
                },
            },
            {
                "id": "one_b.reset",
                "labels": [],
                "layoutOptions": {
                    "org.eclipse.elk.port.side": "WEST",
                },
            },
            {
                "id": "one_b.out",
                "labels": [],
                "layoutOptions": {
                    "org.eclipse.elk.port.side": "EAST",
                },
            },
        ],
    )
    tree.add_node("one_b")
    for id, gate in gates:
        g.add_node(id, **gate.node(id=id).dict())
        tree.add_edge("one_b", id)

    g.add_node("set", layoutOptions=sizing_opts, properties={})
    g.add_node("reset", layoutOptions=sizing_opts, properties={})
    g.add_node("output", layoutOptions=sizing_opts, properties={})

    step_state(states, set_=False, reset=False)
    step_state(states, set_=False, reset=False)
    update_graph(states, g, init=True)

    # configure app
    app = ipyelk.Elk(
        transformer=ipyelk.nx.XELK(layouts={}, source=(g, tree)),
        layout={"height": "100%"},
        style={
            " .on rect.elknode, .on .elkport": {
                "fill": "var(--jp-brand-color2)",
            },
            " .on .elklabel": {
                "fill": "white",
            },
            " .on, .on .elknode": {
                "stroke": "var(--jp-brand-color2)",
                "stroke-width": "4",
                "transition": "all 0.2s",
            },
            " .off, .off .elknode": {
                "transition": "all 0.2s",
            },
        },
    )

    app.diagram.symbols = gate_library

    def step(change):
        g = app.transformer.source[0]
        set_ = "set" in change.new
        reset = "reset" in change.new
        step_state(states, set_, reset)
        step_state(states, set_, reset)
        update_graph(states, g)

        app.refresh()

    app.observe(step, "selected")
    toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app


if __name__ == "__main__":
    app = onebit_memory_example()
    display(app)